In [2]:
import pandas as pd
import numpy as np

In [3]:
data_preprocessed = pd.read_csv('preprocessed_data.csv')

In [4]:
data_preprocessed.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


In [ ]:
## Create Targets

In [6]:
#Find the median in the Absenteeism column
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [19]:
# targets for logistic regression
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] > 
                   data_preprocessed['Absenteeism Time in Hours'].median(), 1, 0)

In [20]:
# New series in the original data frame that will contain the targets for the regression
data_preprocessed['Excessive Absenteeism'] = targets

In [11]:
# Dropping the unnecessary variables
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours','Day of the Week',
                                            'Daily Work Load Average','Distance to Work'],axis=1)
data_with_targets.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,0,0,0,1,7,289,33,30,0,2,1,1
1,0,0,0,0,7,118,50,31,0,1,0,0
2,0,0,0,1,7,179,38,31,0,0,0,0
3,1,0,0,0,7,279,39,24,0,2,0,1
4,0,0,0,1,7,289,33,30,0,2,1,0


In [ ]:
## Select inputs for the regression

In [21]:
inputs = data_with_targets.iloc[:,:-1]

In [ ]:
## Standardize the data using Custom Scaler

In [22]:
from sklearn.preprocessing import StandardScaler
absenteeism_scaler = StandardScaler()

In [23]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler



class CustomScaler(BaseEstimator,TransformerMixin): 
    

    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        
       
        self.scaler = StandardScaler(copy,with_mean,with_std)
        
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    
  
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    

    def transform(self, X, y=None, copy=None):
        
       
        init_col_order = X.columns
        
        
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        

        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
       
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [18]:
inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [ ]:
# Standardize all the inputs apart from the dummy variables

In [28]:
# select the columns to omit
cols_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Education']
cols_to_scale = [x for x in inputs.columns.values if x not in cols_to_omit]

In [30]:
absenteeism_scaler = CustomScaler(cols_to_scale)
absenteeism_scaler.fit(inputs)

C:\Users\91961\anaconda3\lib\site-packages\sklearn\base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CustomScaler(columns=['Month Value', 'Transportation Expense', 'Age',
                      'Body Mass Index', 'Children', 'Pets'],
             copy=None, with_mean=None, with_std=None)

In [31]:
#scaled the inputs
scaled_inputs = absenteeism_scaler.transform(inputs)

In [32]:
scaled_inputs

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.030796,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.030796,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.030796,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.030796,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.030796,1.005844,-0.536062,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.568019,-0.654143,0.562059,-1.114186,1,0.880469,-0.589690
696,1,0,0,0,-0.568019,0.040034,-1.320435,-0.643782,0,-0.019280,1.126663
697,1,0,0,0,-0.568019,1.624567,-1.320435,-0.408580,1,-0.919030,-0.589690
698,0,0,0,1,-0.568019,0.190942,-0.692937,-0.408580,1,-0.919030,-0.589690


In [ ]:
## Split the data into test and train sets

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, #train_size = 0.8, 
                                                                            test_size = 0.2, random_state = 20)

In [35]:
print (x_train.shape, y_train.shape)

(560, 11) (560,)


In [36]:
print (x_test.shape, y_test.shape)

(140, 11) (140,)


In [ ]:
## Using Sklearn for Logistic Regression

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [ ]:
# Train the model

In [38]:
reg = LogisticRegression()

In [39]:
reg.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
# train accuracy of the model
reg.score(x_train,y_train)

0.7767857142857143

In [ ]:
# Finding intercepts and coeff

In [41]:
reg.intercept_

array([-1.60957471])

In [42]:
reg.coef_

array([[ 2.77151176,  0.93168817,  3.09210221,  0.8090592 ,  0.00781237,
         0.62505482, -0.17390339,  0.28829409, -0.24081615,  0.35753531,
        -0.27337422]])

In [44]:
feature_names = inputs.columns.values

In [46]:
# Make a summary table
summary_table = pd.DataFrame (columns=['Feature name'], data = feature_names)
summary_table['Coefficient'] = np.transpose(reg.coef_)
summary_table


,Feature name,Coefficient
0,Reason_1,2.771512
1,Reason_2,0.931688
2,Reason_3,3.092102
3,Reason_4,0.809059
4,Month Value,0.007812
5,Transportation Expense,0.625055
6,Age,-0.173903
7,Body Mass Index,0.288294
8,Education,-0.240816
9,Children,0.357535


In [47]:
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]

summary_table = summary_table.sort_index()
summary_table

,Feature name,Coefficient
0,Intercept,-1.609575
1,Reason_1,2.771512
2,Reason_2,0.931688
3,Reason_3,3.092102
4,Reason_4,0.809059
5,Month Value,0.007812
6,Transportation Expense,0.625055
7,Age,-0.173903
8,Body Mass Index,0.288294
9,Education,-0.240816


In [49]:
# Accuracy after using test data
reg.score(x_test,y_test)

0.7357142857142858